# Building the Car Make and Model Classifier
This notebook demonstrates how to wrap an API call to Algorithmia into a skill. The process is the same for other providers such as Microsoft, Google, Aws, etc.

In [ ]:
from cortex import Cortex
%reload_ext cortex

## Building the car_classifier Action

We will be retrieving our Algorithmia API key from the params via the properties rather than the payload. This allows us to set the property as 'Secure', such that in Cortex Studio it will look for it in the secure vault rather than having it manually input every time.

The flag '--requirements Algorithmia' tells the backend to 'pip install' Algorithmia into the docker container. This flag should be used when libraries must be installed before they can be imported.

Make sure to replace 'your_initials' with your initials to prevent namespace issues.

**Note** 
To use this notebook yourself you need an Algorithmia account/API Key. One can register for free at https://algorithmia.com.

Once your account is activated, click on your profile at the top right and select "My API Keys". Copy the key labeled "default-key".



In [ ]:
%%cortex_action --name 'class/car-classifier-YOUR_INITIALS' --function car_classifier --requirements Algorithmia

from cortex import Message
import Algorithmia

def car_classifier(params):
    msg = Message(params)
    image = msg.payload.get('image_url')
    api_key = msg.properties.get('algorithmia_api_key')
    client = Algorithmia.client(api_key)
    algo = client.algo('LgoBE/CarMakeandModelRecognition/0.4.1')
    algo.set_options(timeout=300)
    return Message.with_payload({'Result': (algo.pipe(image).result)}).to_params()

### Testing Actions
Using the Cortex client, we can test our Action to make sure it deployed properly.

Make sure to replace 'your_initials' with your initials to prevent namespace issues.

In [ ]:
# Instantiate Cortex Client
cortex = Cortex.client()

# Retrieve our Action that was deployed above
# Use the same '<action_namespace>/<action_name>' as you did above for '--name'
action = cortex.action('class/car-classifier-YOUR_INITIALS')

The Action deployment status should say **COMPLETED**.  This will indicate that our Action is ready to invoke.

In [ ]:
action.get_deployment_status()

We are creating a test payload with properties to send to our action for testing. The payload contains a link to an image, and the API Key is stored in the properties.

Make sure to replace 'your_api_key' with the API Key you received from Algorithmia.

In [ ]:
payload_with_properties={
 "payload": {'image_url': 'https://upload.wikimedia.org/wikipedia/commons/4/44/2019_Acura_RDX_A-Spec_front_red_4.2.18.jpg'},
 "properties": {'algorithmia_api_key' : 'YOUR_API_KEY'}
}

invoke_message = Message(payload_with_properties)

Invoke the Action using a Message object.  Here we just pass in the expected _text_ parameter in the Message payload.

In [ ]:
from cortex import Message
rs = action.invoke(invoke_message)
rs.payload

## Building a Cortex Skill
Now that our Action is ready and tested, we can move on to building a Cortex Skill.  In this simple example, our Skill will just pipe an Input to our Action and route the Output back to the caller.

In [ ]:
builder = cortex.builder()

The _builder_ has multiple entry points, we use the _skill_ method here to declare a new "Car Classifier" Skill.  Each _builder_ method returns an instance of the builder so we can chain calls together.  

Make sure to replace 'YOUR_INITIALS' with your initials to prevent namespace issues.

In [ ]:
car_classifier = builder.skill('class/car-classifier-YOUR_INITIALS').title('Car Classifier-YOUR_INITIALS').description('Returns the make and model of a car from a given URL')

Next, we use the Input sub-builder to construct our Skill Input.  This is where we declare how our Input will route messages.  In this simple case, we use the _all_ routing which routes all input messages to same Action for processing and declares wich Output to route Action outputs to.  We pass in our Action from the previous section to wire the Skill to the Action (we could have also passed in the Action name here).  Calling _build_ on the Input will create the input object, add it to the Skill builder, and return the Skill builder.

In [ ]:
car_classifier = car_classifier.input('image-url').title('Image URL').parameter(name='image_url', type='string').all_routing(action, 'result').build()

**Note** It is important to keep track of what you name this property. Not only will the action itself be retrieving it from the params based on the name, but
it will also need to be the same as the name of the key stored in the vault in order to use it. In our vault we have our secure key stored as 'algorithmia_api_key'

In [ ]:
car_classifier = car_classifier.property(name = 'algorithmia_api_key', data_type = 'string', title = 'API Key', description = 'Your API Key from Algorithmia', secure = True)

In the previous step, we referenced an Output called **result**.  We can create that Output here using the Output sub-builder.

In [ ]:
car_classifier = car_classifier.output('result').title('Result').parameter(name='result', type='object').build()

## Preview the CAMEL Document
We can preview the CAMEL document that each builder will create using the _to_camel_ method.

In [ ]:
car_classifier.to_camel()

## Final Build and Publish
We are now ready to build and publish our Skill.

In [ ]:
skill = car_classifier.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))

Make sure to replace 'your_api_key' with the API Key yoou received from Algorithmia

In [ ]:
payload_with_properties={
 "payload": {'image_url': 'https://www.cars.com/cstatic-images/car-pictures/xl/usc70toc041g021001.png'},
 "properties": {'api_key' : 'YOUR_API_KEY'}
}

In [ ]:
rs = skill.invoke(input_name='image-url', message=Message(payload_with_properties))
rs.payload